In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.onnx

# 1. Charger MNIST
train_data = datasets.MNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.MNIST(root="data", train=False, download=True, transform=ToTensor())

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 2. Modèle simple
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = SimpleNN()

# 3. Entraînement
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    for X, y in dataloader:
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# 4. Évaluer
def test(dataloader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            predicted_labels = pred.argmax(dim=1)
            correct += (predicted_labels == y).sum().item()
            total += y.size(0)
    return correct / total

# 5. Entraîner plusieurs epochs
for epoch in range(5):
    loss = train_epoch(train_loader, model, loss_fn, optimizer)
    acc = test(test_loader, model)
    print(f"Epoch {epoch+1} — Loss: {loss:.4f} — Test accuracy: {acc:.4f}")

# 6. Exporter en ONNX
model.eval()
dummy_input = torch.randn(1, 1, 28, 28)
torch.onnx.export(
    model, dummy_input,
    "mnist_model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)
print("Modèle exporté en mnist_model.onnx")


Epoch 1 — Loss: 1.2364 — Test accuracy: 0.8629
Epoch 2 — Loss: 0.4910 — Test accuracy: 0.8925
Epoch 3 — Loss: 0.3921 — Test accuracy: 0.9034
Epoch 4 — Loss: 0.3523 — Test accuracy: 0.9105
Epoch 5 — Loss: 0.3287 — Test accuracy: 0.9152
Modèle exporté en mnist_model.onnx


In [7]:
import base64
import io
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage

# Charger dataset test MNIST
test_data = datasets.MNIST(root="data", train=False, download=True, transform=ToTensor())

# Récupérer une image et label
img_tensor, label = test_data[0]  # image 0
print("Label réel:", label)

# Convertir en image PIL
img_pil = ToPILImage()(img_tensor)

# Sauvegarder en PNG dans un buffer
buffer = io.BytesIO()
img_pil.save(buffer, format="PNG")
img_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

# Générer code HTML img base64 à copier
print(f'<img id="mnist-img" src="data:image/png;base64,{img_base64}" alt="MNIST test image"/>')


Label réel: 7

